In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from MPyDATA_examples.Olesik_et_al_2020.setup import Setup
from MPyDATA_examples.Olesik_et_al_2020.demo_plot_convergence  import plot
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3, x_log_of_pn
from MPyDATA.options import Options
from MPyDATA_examples.Olesik_et_al_2020.simulation import Simulation
# from MPyDATA_examples.Olesik_et_al_2020.analysis import analysis
from MPyDATA_examples.utils.show_plot import show_plot



In [19]:
GCs = np.linspace(.1,.9, 5)
nrs = np.linspace(64,256, 5, dtype = int)

In [20]:
from joblib import Parallel, parallel_backend, delayed

In [27]:
grid_layout = x_p2()
psi_coord = x_p2()
options = Options(n_iters=2,infinite_gauge=False, flux_corrected_transport=False,third_order_terms=False)
setup = Setup()

In [28]:
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA.arakawa_c.discretisation import discretised_analytical_solution
from MPyDATA_examples.utils.error_norms import L2, GMD

In [29]:
def analysis(nr, GC):
    setup = Setup(nr = nr, mixing_ratios_g_kg = [1,4])
    simulation = Simulation(setup, grid_layout,  psi_coord, options, GC)
    simulation.step(simulation.out_steps[-1])
    t = simulation.out_steps[-1] * simulation.dt
    rh = simulation.rh
    pdf_t = PdfEvolver(setup.pdf, setup.drdt, t)
    analytical = discretised_analytical_solution(
                simulation.rh.magnitude,
                lambda r: pdf_t(r * rh.units).magnitude
            ) * pdf_t(rh[0]).units
    numerical = simulation.n
    loc_of_maximum_num = simulation.r[np.argmax(numerical)]
    loc_of_maximum_anal = simulation.r[np.argmax(analytical)]
    maximum_num = np.max(numerical)
    maximum_anal = np.max(analytical)
#     mean = np.mean(numerical)
#     std = np.std(numerical)
    measure_location = (loc_of_maximum_num / loc_of_maximum_anal).magnitude
    measure_height = (maximum_num / maximum_anal).magnitude
    error_L2 = L2(numerical.magnitude, analytical.magnitude, simulation.out_steps[-1])
    error_GMD = np.log(GMD(numerical.magnitude, analytical.magnitude, t.magnitude)) / np.log(2)
    return nr, GC, error_GMD, measure_location, measure_height

In [ ]:
with parallel_backend('threading', n_jobs=-2):
    results0 = Parallel(verbose=10)(
        delayed(analysis)(nr, GC)
        for nr in nrs
        for GC in GCs
    )

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
../..\MPyDATA_examples\Olesik_et_al_2020\setup.py:49: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  )[0] * yunit * xunit ** 4
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-2)]: Done  15 out of  25 | elapsed:  3.5min remaining:  2.3min
[Parallel(n_jobs=-2)]: Done  18 out of  25 | elapsed:  4.2min remaining:  1.6min
[Parallel(n_jobs=-2)]: Done  21 out of  25 | elapsed:  4.8min remaining:   54.4s


In [ ]:
# results = np.array(results).T
results = [list(i) for i in zip(*results0)]
plot_setup = np.array(results[0:2])
measures = {'log$_2$(err)':results[2], 'location ratio':results[3], 'height ratio':results[4]}

In [ ]:
for measure in measures.keys():
    plot(plot_setup[0], plot_setup[1], measures[measure], name = measure, n_levels = 10)
    show_plot(filename = f'convergence_{measure}.pdf')